In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Define device for torch
use_cuda = True
print("CUDA is available:", torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

CUDA is available: True


In [3]:
from datasets import load_dataset

test_dataset = load_dataset("lelexuanzz/Gossipcop_Politifact_Test")
print(test_dataset)

Generating train split: 100%|██████████| 6900/6900 [00:00<00:00, 38734.43 examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'tfidf_embedding', 'bow_embedding', 'w2v_embeddings'],
        num_rows: 6900
    })
})


In [4]:
test_df = test_dataset["train"].to_pandas()
display(test_df)

,text,label,tfidf_embedding,bow_embedding,w2v_embeddings
0,The media reported on “5 TikTok dances you can...,0,"[0.32230591024079835, -0.0283119498671771, 0.5...","[4.111553694916509, -1.409115414997175, -2.222...","[0.023112578, 0.004341462, -0.028385904, 0.039..."
1,"""Over 240,000 'unverified' ballots have alread...",0,"[0.3085783060828312, 0.11941719274360035, 0.04...","[15.340171294779946, 0.6782201912562608, -3.95...","[0.013536842, 0.009806181, 0.0037024287, 0.046..."
2,"Says ""Ron Johnson is making excuses for rioter...",1,"[0.4233043848185321, 0.07862740948902779, 0.05...","[17.57951108926487, -3.370470538016004, -6.032...","[0.023063188, 0.059482925, 0.01734151, 0.06674..."
3,“We have seen over the last 10 years ... under...,0,"[0.3235193218432442, 0.14350347893253146, -0.0...","[13.623137823124136, 2.594215458879296, 1.1975...","[-0.03447485, 0.012470036, 0.040506534, 0.1266..."
4,“I don’t get involved in the hiring and firing...,0,"[0.21714023268740257, -0.01335753130769729, 0....","[11.715673205868844, -3.897763995131928, -0.56...","[-0.039161813, 0.050845865, 0.045167223, -0.00..."
...,...,...,...,...,...
6895,Wedding Album: Dancing with the Stars Pro Lind...,0,"[0.14163919484466825, -0.18715353304800564, -0...","[0.5081918596591114, -0.4770132598956357, 0.04...","[0.007478841, 0.028465271, 0.01225586, 0.05277..."
6896,WATCH: Sneak Peek: Arizona's Furious Alex Atta...,0,"[0.10089287837511274, -0.1537017105814096, -0....","[0.3445347175467952, -0.49345817219713883, 0.0...","[0.03495687, 0.08366224, 0.03197157, 0.0962388..."
6897,Mary Kay Letourneau 'Hopeful' She Can Fix Marr...,0,"[0.30012685395643773, -0.26290611124497976, -0...","[5.587042723074906, -3.0189127373273474, -0.69...","[0.0066373795, 0.04132683, 0.008552303, 0.0507..."
6898,Charlize Theron still upset Aeon Flux didn’t w...,0,"[0.18598375894519942, -0.26601918527230306, -0...","[1.8348212505816772, -2.5021137033708087, 0.45...","[0.032494266, 0.038722403, -0.01410776, 0.0764..."


### Clean Dataset

In [5]:
# #drop nulls
# test_df.dropna(axis=0, inplace=True)
# #drop duplicates
# test_df.drop_duplicates(inplace=True)
# display(test_df)

### Convert to stylo features

In [6]:
# import sys
# import os
# sys.path.append(os.path.abspath("../feat_eng"))

# from feat_eng import has_quotes, has_url, percent_uppercase, frequency_punctuation, percent_whitespace, frequency_words_length, avg_sentence_length



# def extract_stylometric_features(example):
#     text = example['text']
    
#     if text == None:
#         return{
#             "label": example.get("label"),
#             "has_quotes": 0,
#             "has_url": 0,
#             "percent_uppercase": 0.0,
#             "frequency_punctuation": 0,
#             "percent_whitespace": 0.0,
#             "frequency_words_length_15": 0,
#             "frequency_words_length_14": 0,
#             "frequency_words_length_12": 0,
#             "frequency_words_length_11": 0,
#             "avg_sentence_length": 0.0
#         }
    
#     return {
#         "label": example.get("label"),
#         "has_quotes" : has_quotes(text),
#         "has_url": has_url(text),
#         "percent_uppercase": percent_uppercase(text),
#         "frequency_punctuation": frequency_punctuation(text),
#         "percent_whitespace": percent_whitespace(text),
#         "frequency_words_length_15": frequency_words_length(text, 15),
#         "frequency_words_length_14": frequency_words_length(text, 14),
#         "frequency_words_length_12": frequency_words_length(text, 12),
#         "frequency_words_length_11": frequency_words_length(text, 11),
#         "avg_sentence_length": avg_sentence_length(text)  
#     }

In [7]:
# from datasets import Dataset
# #change dataset back to huggingface form


# test_dataset = Dataset.from_pandas(test_df)
# stylo_test = test_dataset.map(extract_stylometric_features)

# # stylo_test = stylo_test.remove_columns(['title', 'roberta_embedding', 'bow_embedding', 'tfidf_embedding', 'w2v_embedding'])
# print(stylo_test)


In [8]:
# # Push to hub
# stylo_test.push_to_hub("lelexuanzz/Gossipcop_Politifact_Test_Stylo")

## Vector Embeddings


### BOW-TFIDF 

In [9]:
#Tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

tfidf_vectorizer = TfidfVectorizer(min_df=0.05, max_df=0.95, stop_words='english', ngram_range=(1,3))
tfidf_embeddings = tfidf_vectorizer.fit_transform(test_df["text"])
print(tfidf_embeddings.shape)


(6900, 1169)


In [10]:
print(tfidf_embeddings)

  (0, 651)	0.17095999921182603
  (0, 898)	0.08547999960591302
  (0, 488)	0.18282260236668418
  (0, 410)	0.25094353635864913
  (0, 987)	0.08752737992315925
  (0, 481)	0.12926393340734607
  (0, 884)	0.09363649008840541
  (0, 531)	0.16655547350541117
  (0, 930)	0.08363177494944153
  (0, 1093)	0.19299719617363553
  (0, 70)	0.10572111171340268
  (0, 1027)	0.2501857363995298
  (0, 397)	0.047357184997965776
  (0, 379)	0.046229405877630685
  (0, 196)	0.04711218863007459
  (0, 373)	0.2883236599478017
  (0, 813)	0.0596010690357604
  (0, 1014)	0.04779232965612915
  (0, 720)	0.09775046970425214
  (0, 43)	0.0978286176308111
  (0, 803)	0.10831420774686594
  (0, 331)	0.0956263254821396
  (0, 1083)	0.0927461442930606
  (0, 3)	0.07012585544380184
  (0, 1091)	0.10640651716020913
  :	:
  (6899, 930)	0.07211581711202476
  (6899, 1064)	0.09382929076926043
  (6899, 259)	0.21210822357526615
  (6899, 696)	0.19807729037440963
  (6899, 934)	0.10303261586779623
  (6899, 746)	0.19555060610158467
  (6899, 1024)	0.

In [11]:
#use truncated svd to reduce dimensions of tfidf embeddings
#instead of PCA as tfidf embeddings are sparse

from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=300, random_state=45)
tfidf_reduced = svd.fit_transform(tfidf_embeddings)
print(tfidf_reduced.shape)
print(tfidf_reduced)

(6900, 300)
[[ 3.22305910e-01 -2.83119499e-02  5.49363491e-01 ... -2.65537990e-02
   3.65719497e-02  4.78281047e-02]
 [ 3.08578306e-01  1.19417193e-01  4.53342969e-02 ... -1.15097457e-02
   3.49680072e-02  2.44266409e-02]
 [ 4.23304385e-01  7.86274095e-02  5.77382921e-02 ... -2.39881400e-02
   1.95589626e-02 -3.41128586e-02]
 ...
 [ 3.00126854e-01 -2.62906111e-01 -8.32331957e-02 ...  1.83975537e-02
   3.69450673e-02 -2.74397293e-04]
 [ 1.85983759e-01 -2.66019185e-01 -9.32809604e-02 ...  2.11396563e-02
  -1.04245299e-02 -2.02165002e-02]
 [ 1.41732095e-01 -1.63308575e-01 -6.91962399e-02 ...  1.08238266e-02
  -3.30433969e-02 -3.59793545e-02]]


In [12]:
# Add a new column to the DataFrame
test_df['tfidf_embedding'] = list(tfidf_reduced)
display(test_df)

,text,label,tfidf_embedding,bow_embedding,w2v_embeddings
0,The media reported on “5 TikTok dances you can...,0,"[0.32230591024075883, -0.028311949900530733, 0...","[4.111553694916509, -1.409115414997175, -2.222...","[0.023112578, 0.004341462, -0.028385904, 0.039..."
1,"""Over 240,000 'unverified' ballots have alread...",0,"[0.3085783060828623, 0.11941719268786223, 0.04...","[15.340171294779946, 0.6782201912562608, -3.95...","[0.013536842, 0.009806181, 0.0037024287, 0.046..."
2,"Says ""Ron Johnson is making excuses for rioter...",1,"[0.42330438481857957, 0.07862740950249068, 0.0...","[17.57951108926487, -3.370470538016004, -6.032...","[0.023063188, 0.059482925, 0.01734151, 0.06674..."
3,“We have seen over the last 10 years ... under...,0,"[0.3235193218432473, 0.1435034790001879, -0.01...","[13.623137823124136, 2.594215458879296, 1.1975...","[-0.03447485, 0.012470036, 0.040506534, 0.1266..."
4,“I don’t get involved in the hiring and firing...,0,"[0.21714023268741092, -0.013357531354942328, 0...","[11.715673205868844, -3.897763995131928, -0.56...","[-0.039161813, 0.050845865, 0.045167223, -0.00..."
...,...,...,...,...,...
6895,Wedding Album: Dancing with the Stars Pro Lind...,0,"[0.14163919484467705, -0.1871535330863532, -0....","[0.5081918596591114, -0.4770132598956357, 0.04...","[0.007478841, 0.028465271, 0.01225586, 0.05277..."
6896,WATCH: Sneak Peek: Arizona's Furious Alex Atta...,0,"[0.10089287837516506, -0.15370171061200275, -0...","[0.3445347175467952, -0.49345817219713883, 0.0...","[0.03495687, 0.08366224, 0.03197157, 0.0962388..."
6897,Mary Kay Letourneau 'Hopeful' She Can Fix Marr...,0,"[0.30012685395650657, -0.2629061112392972, -0....","[5.587042723074906, -3.0189127373273474, -0.69...","[0.0066373795, 0.04132683, 0.008552303, 0.0507..."
6898,Charlize Theron still upset Aeon Flux didn’t w...,0,"[0.18598375894521132, -0.26601918530104635, -0...","[1.8348212505816772, -2.5021137033708087, 0.45...","[0.032494266, 0.038722403, -0.01410776, 0.0764..."


### BOW

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer = CountVectorizer(min_df=0.05, max_df=0.95, stop_words='english', ngram_range=(1,3))
bow_embeddings = bow_vectorizer.fit_transform(test_df["text"])
print(bow_embeddings.shape)

(6900, 1169)


In [14]:
print(bow_embeddings)

  (0, 651)	2
  (0, 898)	1
  (0, 488)	2
  (0, 410)	2
  (0, 987)	1
  (0, 481)	1
  (0, 884)	1
  (0, 531)	3
  (0, 930)	2
  (0, 1093)	2
  (0, 70)	1
  (0, 1027)	3
  (0, 397)	1
  (0, 379)	1
  (0, 196)	1
  (0, 373)	4
  (0, 813)	1
  (0, 1014)	1
  (0, 720)	2
  (0, 43)	2
  (0, 803)	2
  (0, 331)	1
  (0, 1083)	1
  (0, 3)	1
  (0, 1091)	1
  :	:
  (6899, 930)	1
  (6899, 1064)	1
  (6899, 259)	1
  (6899, 696)	1
  (6899, 934)	1
  (6899, 746)	1
  (6899, 1024)	1
  (6899, 1060)	1
  (6899, 1111)	1
  (6899, 626)	1
  (6899, 673)	1
  (6899, 133)	1
  (6899, 747)	1
  (6899, 941)	1
  (6899, 258)	1
  (6899, 1156)	1
  (6899, 878)	2
  (6899, 1011)	1
  (6899, 604)	1
  (6899, 327)	1
  (6899, 946)	1
  (6899, 369)	2
  (6899, 940)	1
  (6899, 425)	1
  (6899, 1009)	1


In [15]:
bow_reduced = svd.fit_transform(bow_embeddings)
print(bow_reduced.shape)
print(bow_reduced)

(6900, 300)
[[ 4.11155369 -1.40911541 -2.22285019 ... -0.05428714 -0.12782605
   0.2925218 ]
 [15.34017129  0.67822019 -3.95217032 ...  1.19092561  1.43619475
  -1.39075334]
 [17.57951109 -3.37047054 -6.03270616 ...  0.35279308 -0.41493208
   0.11685365]
 ...
 [ 5.58704272 -3.01891274 -0.69692462 ... -0.78713293 -0.20372279
  -1.05756739]
 [ 1.83482125 -2.5021137   0.45429264 ... -0.1381487  -0.09905513
  -0.19622964]
 [ 1.03713558 -0.88783167 -0.09315308 ... -0.14861279  0.0564323
   0.29132153]]


In [16]:
# Add a new column to the DataFrame
test_df['bow_embedding'] = list(bow_reduced)
display(test_df)

,text,label,tfidf_embedding,bow_embedding,w2v_embeddings
0,The media reported on “5 TikTok dances you can...,0,"[0.32230591024075883, -0.028311949900530733, 0...","[4.111553694916485, -1.4091154149948346, -2.22...","[0.023112578, 0.004341462, -0.028385904, 0.039..."
1,"""Over 240,000 'unverified' ballots have alread...",0,"[0.3085783060828623, 0.11941719268786223, 0.04...","[15.340171294779958, 0.6782201913070947, -3.95...","[0.013536842, 0.009806181, 0.0037024287, 0.046..."
2,"Says ""Ron Johnson is making excuses for rioter...",1,"[0.42330438481857957, 0.07862740950249068, 0.0...","[17.579511089264834, -3.370470538008803, -6.03...","[0.023063188, 0.059482925, 0.01734151, 0.06674..."
3,“We have seen over the last 10 years ... under...,0,"[0.3235193218432473, 0.1435034790001879, -0.01...","[13.623137823124184, 2.594215458815296, 1.1975...","[-0.03447485, 0.012470036, 0.040506534, 0.1266..."
4,“I don’t get involved in the hiring and firing...,0,"[0.21714023268741092, -0.013357531354942328, 0...","[11.715673205868814, -3.8977639951627094, -0.5...","[-0.039161813, 0.050845865, 0.045167223, -0.00..."
...,...,...,...,...,...
6895,Wedding Album: Dancing with the Stars Pro Lind...,0,"[0.14163919484467705, -0.1871535330863532, -0....","[0.5081918596591085, -0.4770132598925509, 0.04...","[0.007478841, 0.028465271, 0.01225586, 0.05277..."
6896,WATCH: Sneak Peek: Arizona's Furious Alex Atta...,0,"[0.10089287837516506, -0.15370171061200275, -0...","[0.34453471754679216, -0.49345817220335975, 0....","[0.03495687, 0.08366224, 0.03197157, 0.0962388..."
6897,Mary Kay Letourneau 'Hopeful' She Can Fix Marr...,0,"[0.30012685395650657, -0.2629061112392972, -0....","[5.587042723074881, -3.018912737323734, -0.696...","[0.0066373795, 0.04132683, 0.008552303, 0.0507..."
6898,Charlize Theron still upset Aeon Flux didn’t w...,0,"[0.18598375894521132, -0.26601918530104635, -0...","[1.8348212505816643, -2.5021137033474385, 0.45...","[0.032494266, 0.038722403, -0.01410776, 0.0764..."


### CBOW-W2V

In [17]:
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

nltk.download('all')



[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to C:\Users\Lee Le
[nltk_data]    |     Xuan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to C:\Users\Lee Le
[nltk_data]    |     Xuan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Lee Le Xuan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\Lee Le Xuan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\Lee Le Xuan\

True

In [18]:
texts = test_df["text"].astype(str).tolist()

def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [t for t in tokens if t.isalpha() and t not in ENGLISH_STOP_WORDS]
    return tokens

tokenized_texts = [preprocess(t) for t in texts]


In [19]:
from gensim.models import KeyedVectors
import gensim.downloader

#load pretrained model
w2v_vectors = gensim.downloader.load('word2vec-google-news-300')


In [20]:
# # Get top 10,000 most frequent words in corpus
# from collections import Counter

# word_freq = Counter([word for tokens in tokenized_texts for word in tokens])
# vocab = [word for word, _ in word_freq.most_common(10000)]
# word_to_idx = {word: idx for idx, word in enumerate(vocab)}


In [21]:
import numpy as np

embedding_dim = w2v_vectors.vector_size
num_docs = len(tokenized_texts)

w2v_embeddings = np.zeros((num_docs, embedding_dim), dtype=np.float32)

for i, doc in enumerate(tokenized_texts):
    vectors = [w2v_vectors[word] for word in doc if word in w2v_vectors]
    if vectors:
        w2v_embeddings[i] = np.mean(vectors, axis=0)
    else:
        w2v_embeddings[i] = np.zeros(embedding_dim)

In [22]:
print(w2v_embeddings.shape)
test_df['w2v_embedding'] = list(w2v_embeddings)
display(test_df)

(6900, 300)


,text,label,tfidf_embedding,bow_embedding,w2v_embeddings,w2v_embedding
0,The media reported on “5 TikTok dances you can...,0,"[0.32230591024075883, -0.028311949900530733, 0...","[4.111553694916485, -1.4091154149948346, -2.22...","[0.023112578, 0.004341462, -0.028385904, 0.039...","[0.023112578, 0.004341462, -0.028385904, 0.039..."
1,"""Over 240,000 'unverified' ballots have alread...",0,"[0.3085783060828623, 0.11941719268786223, 0.04...","[15.340171294779958, 0.6782201913070947, -3.95...","[0.013536842, 0.009806181, 0.0037024287, 0.046...","[0.013536842, 0.009806181, 0.0037024287, 0.046..."
2,"Says ""Ron Johnson is making excuses for rioter...",1,"[0.42330438481857957, 0.07862740950249068, 0.0...","[17.579511089264834, -3.370470538008803, -6.03...","[0.023063188, 0.059482925, 0.01734151, 0.06674...","[0.023063188, 0.059482925, 0.01734151, 0.06674..."
3,“We have seen over the last 10 years ... under...,0,"[0.3235193218432473, 0.1435034790001879, -0.01...","[13.623137823124184, 2.594215458815296, 1.1975...","[-0.03447485, 0.012470036, 0.040506534, 0.1266...","[-0.03447485, 0.012470036, 0.040506534, 0.1266..."
4,“I don’t get involved in the hiring and firing...,0,"[0.21714023268741092, -0.013357531354942328, 0...","[11.715673205868814, -3.8977639951627094, -0.5...","[-0.039161813, 0.050845865, 0.045167223, -0.00...","[-0.039161813, 0.050845865, 0.045167223, -0.00..."
...,...,...,...,...,...,...
6895,Wedding Album: Dancing with the Stars Pro Lind...,0,"[0.14163919484467705, -0.1871535330863532, -0....","[0.5081918596591085, -0.4770132598925509, 0.04...","[0.007478841, 0.028465271, 0.01225586, 0.05277...","[0.007478841, 0.028465271, 0.01225586, 0.05277..."
6896,WATCH: Sneak Peek: Arizona's Furious Alex Atta...,0,"[0.10089287837516506, -0.15370171061200275, -0...","[0.34453471754679216, -0.49345817220335975, 0....","[0.03495687, 0.08366224, 0.03197157, 0.0962388...","[0.03495687, 0.08366224, 0.03197157, 0.0962388..."
6897,Mary Kay Letourneau 'Hopeful' She Can Fix Marr...,0,"[0.30012685395650657, -0.2629061112392972, -0....","[5.587042723074881, -3.018912737323734, -0.696...","[0.0066373795, 0.04132683, 0.008552303, 0.0507...","[0.0066373795, 0.04132683, 0.008552303, 0.0507..."
6898,Charlize Theron still upset Aeon Flux didn’t w...,0,"[0.18598375894521132, -0.26601918530104635, -0...","[1.8348212505816643, -2.5021137033474385, 0.45...","[0.032494266, 0.038722403, -0.01410776, 0.0764...","[0.032494266, 0.038722403, -0.01410776, 0.0764..."


### Roberta

In [23]:
from transformers import RobertaTokenizer, RobertaModel

roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
roberta_model = RobertaModel.from_pretrained("roberta-base")
roberta_model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [24]:
def get_roberta_embedding(text):
    inputs = roberta_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = roberta_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()


In [25]:
batch_size = 32
roberta_embeddings = []
for i in range(0, len(test_df), batch_size):
    batch_texts = test_df['text'][i:i+batch_size].tolist()
    batch_embeds = [get_roberta_embedding(text) for text in batch_texts]
    roberta_embeddings.extend(batch_embeds)
    print(f"Processed RoBERTa batch {i//batch_size + 1}/{len(test_df)//batch_size + 1}")


Processed RoBERTa batch 1/216
Processed RoBERTa batch 2/216
Processed RoBERTa batch 3/216
Processed RoBERTa batch 4/216
Processed RoBERTa batch 5/216
Processed RoBERTa batch 6/216
Processed RoBERTa batch 7/216
Processed RoBERTa batch 8/216
Processed RoBERTa batch 9/216
Processed RoBERTa batch 10/216
Processed RoBERTa batch 11/216
Processed RoBERTa batch 12/216
Processed RoBERTa batch 13/216
Processed RoBERTa batch 14/216
Processed RoBERTa batch 15/216
Processed RoBERTa batch 16/216
Processed RoBERTa batch 17/216
Processed RoBERTa batch 18/216
Processed RoBERTa batch 19/216
Processed RoBERTa batch 20/216
Processed RoBERTa batch 21/216
Processed RoBERTa batch 22/216
Processed RoBERTa batch 23/216
Processed RoBERTa batch 24/216
Processed RoBERTa batch 25/216
Processed RoBERTa batch 26/216
Processed RoBERTa batch 27/216
Processed RoBERTa batch 28/216
Processed RoBERTa batch 29/216
Processed RoBERTa batch 30/216
Processed RoBERTa batch 31/216
Processed RoBERTa batch 32/216
Processed RoBERTa

In [26]:
roberta_embeddings = np.array(roberta_embeddings)
print(roberta_embeddings.shape)

(6900, 768)


In [27]:
#reduce dimensionality with PCA
from sklearn.decomposition import PCA

pca = PCA(n_components=300, random_state=45)  
roberta_reduced = pca.fit_transform(roberta_embeddings)
print(roberta_reduced.shape)


(6900, 300)


In [28]:
test_df['roberta_embedding'] = list(roberta_reduced)
display(test_df)

,text,label,tfidf_embedding,bow_embedding,w2v_embeddings,w2v_embedding,roberta_embedding
0,The media reported on “5 TikTok dances you can...,0,"[0.32230591024075883, -0.028311949900530733, 0...","[4.111553694916485, -1.4091154149948346, -2.22...","[0.023112578, 0.004341462, -0.028385904, 0.039...","[0.023112578, 0.004341462, -0.028385904, 0.039...","[-1.7613875, -0.29602507, 1.0549823, 0.4420643..."
1,"""Over 240,000 'unverified' ballots have alread...",0,"[0.3085783060828623, 0.11941719268786223, 0.04...","[15.340171294779958, 0.6782201913070947, -3.95...","[0.013536842, 0.009806181, 0.0037024287, 0.046...","[0.013536842, 0.009806181, 0.0037024287, 0.046...","[0.5113651, -0.5632596, -0.08679409, 0.0578204..."
2,"Says ""Ron Johnson is making excuses for rioter...",1,"[0.42330438481857957, 0.07862740950249068, 0.0...","[17.579511089264834, -3.370470538008803, -6.03...","[0.023063188, 0.059482925, 0.01734151, 0.06674...","[0.023063188, 0.059482925, 0.01734151, 0.06674...","[0.4061544, 0.033350937, -0.30923274, -0.04415..."
3,“We have seen over the last 10 years ... under...,0,"[0.3235193218432473, 0.1435034790001879, -0.01...","[13.623137823124184, 2.594215458815296, 1.1975...","[-0.03447485, 0.012470036, 0.040506534, 0.1266...","[-0.03447485, 0.012470036, 0.040506534, 0.1266...","[0.74233645, -0.8022974, 0.30516905, -0.150992..."
4,“I don’t get involved in the hiring and firing...,0,"[0.21714023268741092, -0.013357531354942328, 0...","[11.715673205868814, -3.8977639951627094, -0.5...","[-0.039161813, 0.050845865, 0.045167223, -0.00...","[-0.039161813, 0.050845865, 0.045167223, -0.00...","[0.4902299, -0.04709744, -0.11096978, -0.33383..."
...,...,...,...,...,...,...,...
6895,Wedding Album: Dancing with the Stars Pro Lind...,0,"[0.14163919484467705, -0.1871535330863532, -0....","[0.5081918596591085, -0.4770132598925509, 0.04...","[0.007478841, 0.028465271, 0.01225586, 0.05277...","[0.007478841, 0.028465271, 0.01225586, 0.05277...","[-4.3445463, -0.82720375, -0.76975673, 0.55827..."
6896,WATCH: Sneak Peek: Arizona's Furious Alex Atta...,0,"[0.10089287837516506, -0.15370171061200275, -0...","[0.34453471754679216, -0.49345817220335975, 0....","[0.03495687, 0.08366224, 0.03197157, 0.0962388...","[0.03495687, 0.08366224, 0.03197157, 0.0962388...","[-3.484783, -0.13741834, -0.2212331, 0.3501901..."
6897,Mary Kay Letourneau 'Hopeful' She Can Fix Marr...,0,"[0.30012685395650657, -0.2629061112392972, -0....","[5.587042723074881, -3.018912737323734, -0.696...","[0.0066373795, 0.04132683, 0.008552303, 0.0507...","[0.0066373795, 0.04132683, 0.008552303, 0.0507...","[0.060427, 0.93066996, -0.5383188, -0.11787002..."
6898,Charlize Theron still upset Aeon Flux didn’t w...,0,"[0.18598375894521132, -0.26601918530104635, -0...","[1.8348212505816643, -2.5021137033474385, 0.45...","[0.032494266, 0.038722403, -0.01410776, 0.0764...","[0.032494266, 0.038722403, -0.01410776, 0.0764...","[0.29304072, 1.2304765, -0.0635877, 0.6557827,..."


## Push to Huggingface

In [35]:
from datasets import Dataset

display(test_df)

test_dataset = Dataset.from_pandas(test_df)
print(test_dataset)

,text,label,tfidf_embedding,bow_embedding,w2v_embedding,roberta_embedding
0,The media reported on “5 TikTok dances you can...,0,"[0.32230591024075883, -0.028311949900530733, 0...","[4.111553694916485, -1.4091154149948346, -2.22...","[0.023112578, 0.004341462, -0.028385904, 0.039...","[-1.7613875, -0.29602507, 1.0549823, 0.4420643..."
1,"""Over 240,000 'unverified' ballots have alread...",0,"[0.3085783060828623, 0.11941719268786223, 0.04...","[15.340171294779958, 0.6782201913070947, -3.95...","[0.013536842, 0.009806181, 0.0037024287, 0.046...","[0.5113651, -0.5632596, -0.08679409, 0.0578204..."
2,"Says ""Ron Johnson is making excuses for rioter...",1,"[0.42330438481857957, 0.07862740950249068, 0.0...","[17.579511089264834, -3.370470538008803, -6.03...","[0.023063188, 0.059482925, 0.01734151, 0.06674...","[0.4061544, 0.033350937, -0.30923274, -0.04415..."
3,“We have seen over the last 10 years ... under...,0,"[0.3235193218432473, 0.1435034790001879, -0.01...","[13.623137823124184, 2.594215458815296, 1.1975...","[-0.03447485, 0.012470036, 0.040506534, 0.1266...","[0.74233645, -0.8022974, 0.30516905, -0.150992..."
4,“I don’t get involved in the hiring and firing...,0,"[0.21714023268741092, -0.013357531354942328, 0...","[11.715673205868814, -3.8977639951627094, -0.5...","[-0.039161813, 0.050845865, 0.045167223, -0.00...","[0.4902299, -0.04709744, -0.11096978, -0.33383..."
...,...,...,...,...,...,...
6895,Wedding Album: Dancing with the Stars Pro Lind...,0,"[0.14163919484467705, -0.1871535330863532, -0....","[0.5081918596591085, -0.4770132598925509, 0.04...","[0.007478841, 0.028465271, 0.01225586, 0.05277...","[-4.3445463, -0.82720375, -0.76975673, 0.55827..."
6896,WATCH: Sneak Peek: Arizona's Furious Alex Atta...,0,"[0.10089287837516506, -0.15370171061200275, -0...","[0.34453471754679216, -0.49345817220335975, 0....","[0.03495687, 0.08366224, 0.03197157, 0.0962388...","[-3.484783, -0.13741834, -0.2212331, 0.3501901..."
6897,Mary Kay Letourneau 'Hopeful' She Can Fix Marr...,0,"[0.30012685395650657, -0.2629061112392972, -0....","[5.587042723074881, -3.018912737323734, -0.696...","[0.0066373795, 0.04132683, 0.008552303, 0.0507...","[0.060427, 0.93066996, -0.5383188, -0.11787002..."
6898,Charlize Theron still upset Aeon Flux didn’t w...,0,"[0.18598375894521132, -0.26601918530104635, -0...","[1.8348212505816643, -2.5021137033474385, 0.45...","[0.032494266, 0.038722403, -0.01410776, 0.0764...","[0.29304072, 1.2304765, -0.0635877, 0.6557827,..."


Dataset({
    features: ['text', 'label', 'tfidf_embedding', 'bow_embedding', 'w2v_embedding', 'roberta_embedding'],
    num_rows: 6900
})


In [36]:
test_dataset.push_to_hub("lelexuanzz/Gossipcop_Politifact_Test", commit_message="Added roBERTa embeddings")

Uploading the dataset shards: 100%|██████████| 1/1 [00:17<00:00, 17.41s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/lelexuanzz/Gossipcop_Politifact_Test/commit/ae6f4376cc04e121873e5bae8beb733f740442d1', commit_message='Added roBERTa embeddings', commit_description='', oid='ae6f4376cc04e121873e5bae8beb733f740442d1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/lelexuanzz/Gossipcop_Politifact_Test', endpoint='https://huggingface.co', repo_type='dataset', repo_id='lelexuanzz/Gossipcop_Politifact_Test'), pr_revision=None, pr_num=None)